# connecting to the drive

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!ls

~  data  datalab  drive  neg.txt  nltk_data  pos.txt  so3_lexicons.pickle


In [0]:
!cp drive/DeepLearning/*.txt ./    # copy the data from the drive to the vm 

In [11]:
!ls

~  data  datalab  drive  neg.txt  nltk_data  pos.txt  so3_lexicons.pickle


In [0]:
!pip freeze

# sentiment

In [0]:
import nltk

In [0]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from collections import Counter
import random
import pickle

In [0]:
lemmatizer = WordNetLemmatizer()

In [0]:
hm_lines = 10000000

In [0]:
# need to create the lexicons

def create_lexicons(pos, neg):
  lexicons = []
  for fi in [pos, neg]:
    with open(fi, 'r') as f:
      contents = f.readlines()
      for line in contents[:hm_lines]:
        all_words = word_tokenize(line.lower())
        lexicons += list(all_words)
        
  lexicons = [lemmatizer.lemmatize(l) for l in lexicons]
  w_counts = Counter(lexicons)
  
  lexcon = []
  
  for w in w_counts:
    if 1000 > w_counts[w] > 50:
      lexcon.append(w)
  print('number of lexicons found : ', len(lexcon))
  return(lexcon)

In [0]:
# This function is responsible for featching the data from the source and make samples

def sample_handling(sample, lexicon, classification):
  featureset = []
  with open(sample, 'r') as f:
    contents = f.readlines()
    for l in contents[:hm_lines]:
      current_words = word_tokenize(l.lower())
      current_words = [lemmatizer.lemmatize(w) for w in current_words]
      features = np.zeros(len(lexicon))
      for word in current_words:
        if word.lower() in lexicon:
          word_indx = lexicon.index(word.lower())
          features[word_indx] = 1
          
      features = list(features)
      featureset.append([features, classification])
      
      
  return featureset

In [0]:
# creating the set of data to train and test out model that we'll be creating 
def create_feature_sets_and_labels(pos, neg, test_size=0.1):
  lexicon = create_lexicons(pos, neg)
  features = []
  features += sample_handling('pos.txt', lexicon, [1, 0])
  features += sample_handling('neg.txt', lexicon, [0, 1])
  random.shuffle(features)
  
  features = np.array(features)
  
  testing_size = int(test_size*len(features))
  
  train_x = list(features[:,0][:-testing_size])
  train_y = list(features[:,1][:-testing_size])
  
  test_x = list(features[:,0][-testing_size:])
  test_y = list(features[:,1][-testing_size:])
  
  return train_x, train_y, test_x, test_y

In [19]:
# this is the main cell where we call function to actually create the sample data
# and store the generated data to pickled file

train_x, train_y, test_x, test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')
with open('so3_lexicons.pickle', 'wb') as pk:
  pickle.dump([train_x, train_y, test_x, test_y], pk)

number of lexicons found :  423


In [31]:
!ls

~  data  datalab  drive  neg.txt  nltk_data  pos.txt  so3_lexicons.pickle


In [32]:
!ls -la    # look at the last entry in the out put `so3_lexicons.pickle`

total 137948
drwxr-xr-x  3 root root      4096 Feb  3 16:19 ~
drwxr-xr-x  1 root root      4096 Feb  3 22:26 .
drwxr-xr-x  1 root root      4096 Feb  3 15:41 ..
drwx------  4 root root      4096 Feb  3 15:42 .cache
drwxr-xr-x  3 root root      4096 Feb  3 15:42 .config
drwxr-xr-x  2 root root      4096 Feb  3 16:21 data
drwxr-xr-x  1 root root      4096 Feb  3 18:07 datalab
drwxr-xr-x  2 root root      4096 Feb  3 18:08 drive
drwxr-xr-x  4 root root      4096 Feb  3 15:41 .forever
drwx------  3 root root      4096 Feb  3 18:07 .gdfuse
drwxr-xr-x  5 root root      4096 Feb  3 15:42 .ipython
drwxr-xr-x  2 root root      4096 Feb  3 15:46 .keras
drwx------  3 root root      4096 Feb  3 15:42 .local
-rw-r--r--  1 root root    612459 Feb  3 19:29 neg.txt
drwxr-xr-x 12 root root      4096 Feb  3 21:29 nltk_data
drwx------  3 root root      4096 Feb  3 16:02 .nv
-rw-r--r--  1 root root    626349 Feb  3 19:29 pos.txt
-rw-------  1 root root      1024 Feb  3 15:41 .rnd
-rw-r--r--  1 root root 1

# Deep learining on the above sample generated

In [20]:
import tensorflow as tf

train_x, train_y, test_x, test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

# number of classes : only 2 (positive and negetive sentiment)
n_classes = 2

# batch size is the number of images in each iteration that the network will be able to see 
batch_size = 100

# shape of our input array which is size of lexicon or length of single input train_x[0]
input_shape = len(train_x[0])

# we'll be streaching out whole matrix to work as a single dimensional array

# lets define the variables :
# input : x and output : y
x = tf.placeholder('float', [None, input_shape])
y = tf.placeholder('float', [None, 2])

number of lexicons found :  423


In [0]:
def neural_network_model(input_data):
  hidden_layer1 = {'weights': tf.Variable(tf.random_normal([input_shape, n_nodes_hl1])), 
                   'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
  
  
  hidden_layer2 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])), 
                   'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
  
  
  hidden_layer3 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])), 
                   'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
  
  
  output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])), 
                   'biases': tf.Variable(tf.random_normal([n_classes]))}
  
  # model for each layer
  # input_data * weights + biases = result for next layer
  
  l1 = tf.add(tf.matmul(input_data, hidden_layer1['weights']), hidden_layer1['biases'])
  l1 = tf.nn.relu(l1)
  
  l2 = tf.add(tf.matmul(l1, hidden_layer2['weights']), hidden_layer2['biases'])
  l2 = tf.nn.relu(l2)

  l3 = tf.add(tf.matmul(l2, hidden_layer3['weights']), hidden_layer3['biases'])
  l3 = tf.nn.relu(l3)
  
  output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
  
  return output

In [0]:

# this is the training function that will be very handy we'll use the output of the model here
def train_neural_network(x):
  prediction = neural_network_model(x)
  
  # we need to minimize the cost as it denotes the difference between the prediction that we are making
  # and the actual lable
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
  
  # need to optimize this
  optimizer = tf.train.AdamOptimizer().minimize(cost)
  
  n_epoch = 20
  
  with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    for epoch in range(n_epoch):
      e_loss = 0
      
      '''
      # these two lines were very specific to mnist data
      for _ in range(mnist.train.num_examples//batch_size):
        e_x, e_y = mnist.train.next_batch(batch_size)
        
      '''
      
      idx = 0
      while idx < len(train_x):
        init = idx
        end = idx + batch_size
        idx = end
        
        batch_x = np.array(train_x[init:end])
        batch_y = np.array(train_y[init:end])
        
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
        e_loss += c
        
      print("Epoch_loss : ",e_loss)
      
      
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    print('Accuracy : ', accuracy.eval({x: test_x, y: test_y}))

train_neural_network(x)



In [1]:
!ls

~  data  datalab  drive  neg.txt  nltk_data  pos.txt  so3_lexicons.pickle


In [0]:
!cp so3_lexicons.pickle drive/DeepLearning/